In [1]:
import pandas as pd
A = pd.DataFrame([[1,2],[3,4]], columns=['alpha1','alpha2'])

In [31]:
import pandas as pd

# create a dataframe with sample data
df = pd.DataFrame({'alpha1': [1, 2, 3], 'alpha2': [2, 4, 6], 'Return': [3, 6, 9]})

# access the 'str' attribute of the 'columns' object of the dataframe
df.columns.str

In [23]:
from utils import *

In [32]:
df['tradedate'] = '2023-7-23'

In [36]:
data = pd.DataFrame({
    'tradedate': ['2021-01-01', '2021-01-01', '2021-01-01', '2021-01-01', '2021-01-01', '2021-01-02', '2021-01-02', '2021-01-02', '2021-01-02', '2021-01-02'],
    'alpha1': [1.2, 2.3, 3.4, 4.5, 5.6, 6.7, 7.8, 8.9, 9.0, 10.1],
    'alpha2': [2.1, 3.2, 4.3, 5.4, 6.5, 7.6, 8.7, 9.8, 10.9, 11.0],
    'ind': ['A', 'B', 'C', 'D', 'E', 'A', 'B', 'C', 'D', 'E'],
    'size': ['S', 'S', 'M', 'M', 'L', 'S', 'S', 'M', 'M', 'L'],
    'ret': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
})

In [45]:
def df_qcut(df: pd.DataFrame, n_groups: int = 5):
    """Quantile-based discretization function for pandas.DataFrame."""
    if df.empty or df.count().sum() == 0:
        return pd.DataFrame.reindex_like(df)
    
    def q_cut(x: pd.Series, n_groups: int = n_groups):
        try:
            return pd.qcut(x, n_groups, duplicates="drop", labels = ['p' + str(i) for i in range(1, n_groups + 1)])
        except:
            try:
                return pd.qcut(x.rank(method='first'),
                                      q=np.linespace(0, 1, num=n_groups + 1),
                                      duplicates='drop',
                                      labels=['p' + str(i) for i in range(1, n_groups + 1)])
            except:
                try:
                    return pd.qcut(x.rank(method='first'), q=n_groups, duplicates='drop', labels=['p' + str(i) for i in range(1, n_groups + 1)])
                except ValueError as e:
                    _, n_bins = pd.qcut(x.rank(method='first'), q=n_groups, duplicates='drop', retbins=True, labels=False)
                    if len(n_bins) == 0:
                        return pd.qcut(x.rank(method='first'), q=n_groups, duplicates='drop', labels=['p' + str(i) for i in range(1, len(n_bins))])
                    else:
                        print(x)
                        raise ValueError(len(x.unique()), repr(e))
    return df.apply(q_cut)

In [70]:
s1 = pd.Series(range(1,10))
s2 = pd.Series([1,1,1,1,1,1,1,1,10])
s3 = pd.Series([1,3,1,4,1,1,6,1,10])
s4 = pd.Series([1,1,1,1,1,1,1,1,1])
df = pd.DataFrame({'s1':s1,'s2':s2,'s3':s3,'s4':s4})

In [71]:
res = df_qcut(df, 5)

In [86]:
factor_data = pd.DataFrame({'tradedate': pd.date_range('2023-07-10', '2023-07-23'),
                            'stock':['A','B','A','B','A','B','A','B','A','B','A','B','A','B'],
                            'factor': [1,2,3,4,5,6,7,8,9,10,11,12,13,14]}).set_index(['tradedate','stock'])

In [88]:
temp = factor_data.swaplevel(axis=0).sort_index(axis=0)
temp

factor
stock tradedate         
A     2023-07-10       1
      2023-07-12       3
      2023-07-14       5
      2023-07-16       7
      2023-07-18       9
      2023-07-20      11
      2023-07-22      13
B     2023-07-11       2
      2023-07-13       4
      2023-07-15       6
      2023-07-17       8
      2023-07-19      10
      2023-07-21      12
      2023-07-23      14

In [101]:
date_idx = pd.DatetimeIndex(temp.index.get_level_values('tradedate').unique())

In [106]:
def resample(data: pd.DataFrame, freq: str = 'M'):
    date_idx = pd.DatetimeIndex(data.index.get_level_values('tradedate').unique())
    year = date_idx.year
    if freq == 'D':
        freq = date_idx.dayofyear
    elif freq == 'Y':
        freq = date_idx.year
    elif freq == 'M':
        freq = date_idx.month
    elif freq == 'W':
        year = date_idx.isocalendar().year  #每年的最后一天不一定是当年（有可能是下一年）的星期，最开始一天不一定是当年（有可能是上一年）的星期
        freq = date_idx.isocalendar().week
    elif freq == 'Q':
        freq = date_idx.quarter
    date_idx = date_idx.to_frame().groupby([year, freq], as_index=False).last().set_index('tradedate')

    return data.reindex(date_idx.index, level='tradedate', method='ffill')

In [110]:
date_idx.dayofyear

Int64Index([191, 193, 195, 197, 199, 201, 203, 192, 194, 196, 198, 200, 202,
            204],
           dtype='int64', name='tradedate')

In [115]:
freq = date_idx.dayofyear

In [1]:
from config import C

In [5]:
_global_conig = C.__dict__['_config']